In [1]:
import torch
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [2]:
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(degrees=40),        # 🔄 different angles
    transforms.RandomHorizontalFlip(p=0.5),       # ↔ mirror
    transforms.RandomVerticalFlip(p=0.5),         # ↕ mirror
    transforms.ColorJitter(
        brightness=0.2,
        contrast=0.2,
        saturation=0.2
    ),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [3]:
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [5]:
train_dataset = ImageFolder("dataset/train", transform=train_transforms)
test_dataset  = ImageFolder("dataset/test", transform=test_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)

class_names = train_dataset.classes
print("Classes:", class_names)

Classes: ['apples', 'avocados', 'bananas', 'grapes', 'guava', 'limes', 'mangos', 'oranges', 'pineapples', 'watermelons']


In [6]:
import torch.nn as nn

In [7]:
class FruitCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 28 * 28, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = FruitCNN(num_classes=len(class_names)).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [9]:
print(device)

cuda


In [15]:
epochs = 100

for epoch in range(epochs):
    model.train()
    running_loss = 0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    if (epoch+1) % 10 == 0:
        torch.save(model.state_dict(), "fruit_classifier.pth")
        
    train_acc = 100 * correct / total

    print(f"Epoch [{epoch+1}/{epochs}] "
          f"Loss: {running_loss:.4f} "
          f"Train Acc: {train_acc:.2f}%")

Epoch [1/100] Loss: 3.0185 Train Acc: 88.06%
Epoch [2/100] Loss: 3.6940 Train Acc: 85.45%
Epoch [3/100] Loss: 2.5096 Train Acc: 91.42%
Epoch [4/100] Loss: 2.9192 Train Acc: 87.31%
Epoch [5/100] Loss: 2.4475 Train Acc: 89.55%
Epoch [6/100] Loss: 2.5310 Train Acc: 89.93%
Epoch [7/100] Loss: 2.2772 Train Acc: 91.42%
Epoch [8/100] Loss: 2.8326 Train Acc: 89.55%
Epoch [9/100] Loss: 3.4529 Train Acc: 86.57%
Epoch [10/100] Loss: 3.2448 Train Acc: 88.81%
Epoch [11/100] Loss: 3.1034 Train Acc: 86.94%
Epoch [12/100] Loss: 2.2733 Train Acc: 92.16%
Epoch [13/100] Loss: 2.7049 Train Acc: 88.06%
Epoch [14/100] Loss: 2.4670 Train Acc: 89.18%
Epoch [15/100] Loss: 2.9760 Train Acc: 90.67%
Epoch [16/100] Loss: 2.4227 Train Acc: 91.42%
Epoch [17/100] Loss: 2.8210 Train Acc: 90.67%
Epoch [18/100] Loss: 2.5752 Train Acc: 91.42%
Epoch [19/100] Loss: 2.1883 Train Acc: 92.91%
Epoch [20/100] Loss: 2.6455 Train Acc: 88.81%
Epoch [21/100] Loss: 1.4306 Train Acc: 94.03%
Epoch [22/100] Loss: 2.1554 Train Acc: 89.9

In [16]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print(f"✅ Test Accuracy: {100 * correct / total:.2f}%")

✅ Test Accuracy: 83.33%


In [17]:
torch.save(model.state_dict(), "fruit_classifier.pth")